In [1]:
import requests 
from bs4 import BeautifulSoup 
from bs4 import UnicodeDammit

atm_url = 'https://www.trustpilot.com/categories/atm'

response = requests.get(atm_url)

soup = BeautifulSoup(response.content, "html.parser")

In [2]:
best_atm = soup.find_all('div', attrs = {'class': "paper_paper__1PY90 paper_outline__lwsUX card_card__lQWDv card_noPadding__D8PcU styles_wrapper__2JOo2"})
com1 = best_atm[0]
name1 = com1.find('p', {'class': 'typography_heading-xs__jSwUz typography_appearance-default__AAY17 styles_displayName__GOhL2'}).text
print(name1)
score = com1.find('p', {'class': 'typography_body-m__xgxZ_ typography_appearance-subtle__8_H2l styles_ratingText__yQ5S7'}).text
rating1 = score.split(" ")[1].split("|")[0]
review1 = score.split(" ")[1].split("|")[1]
print(rating1)
print(review1)

domain1 = com1.find('div', {'class': 'styles_wrapper___E6__ styles_categoriesLabels__FiWQ4 styles_desktop__U5iWw'}).text
print(domain1)

Evergreen Credit Union
4.9
216
ATM·Credit Union·Financial Institution


In [3]:
import pandas as pd

name = []
trustscore = []
num_review = []
domain = []

for company in best_atm:
    
    name.append(company.find('p', {'class': 'typography_heading-xs__jSwUz typography_appearance-default__AAY17 styles_displayName__GOhL2'}).text)
    
    try:
        trustscore_text = company.find('span', {'class': 'typography_body-m__xgxZ_ typography_appearance-subtle__8_H2l styles_trustScore__8emxJ'}).text
        trustscore_value = trustscore_text.split()[1]
        trustscore.append(trustscore_value)
    except AttributeError:
        trustscore.append(None)
    
    try:
        num_review_text = company.find('p', {'class': 'typography_body-m__xgxZ_ typography_appearance-subtle__8_H2l styles_ratingText__yQ5S7'}).text
        num_review_value = num_review_text.split('|')[1].split()[0]
        num_review.append(num_review_value)
    except AttributeError:
        num_review.append(None)
    
    domain.append(company.find('div', {'class': 'styles_wrapper___E6__ styles_categoriesLabels__FiWQ4 styles_desktop__U5iWw'}).text)

    
atm_df = pd.DataFrame(list(zip(name, trustscore, num_review, domain)), columns=['Company name', 'Trustscore', 'Number of Reviews', 'Domain'])
atm_df.head()

atm_df.to_csv('atm.csv', index=False)

In [4]:
atm_df.head(10)

,Company name,Trustscore,Number of Reviews,Domain
0,Evergreen Credit Union,4.9,216,ATM·Credit Union·Financial Institution
1,Liberty First Credit Union,4.8,196,Credit Union·Financial Institution·Car Finance...
2,Crypto Dispensers,2.3,119,Non-Bank Financial Service·ATM·Cryptocurrency ...
3,Swadesh,4.3,7,Bank·Money Transfer Service·ATM·Financial Inst...
4,Heritage Valley Federal Credit Union,4.1,11,ATM·Bank·Federal Credit Union·Financial Instit...
5,The Payment HQ,3.8,2,Business to Business Service·Payment Service·F...
6,ASDFTEST,3.7,1,ATM
7,Kryptominerstech,3.7,1,Money Order Service·Business Center·Gold Minin...
8,WPC Merchant Services & Credit Card Processing,3.7,1,ATM·Business to Business Service
9,Meriwest Credit Union,3.5,8,Credit Counseling Service·Car Finance and Loan...
